## **PyTorch Going Modular**

In this we will basically talk about, *how to turn the notebook code into python scripts?*

#### Going Modular

It involves turning notebook code into a series of different python scripts that offer similar functionality. For example, we can trun our notebook code from a series of cells into the followinh python files:

-  `data_setup.py` - a file to prepare and download data if needed.
- `engine.py` - a file containing various training functions
- `model_builder.py` or `model.py` - a file to create a PyTorch model
- `train.py` - a file to leverage all other files and train a target PyTorch model
- `utils.py` - a file dedicated to helpful utility functions

#### Why go modular?

Notebooks are nice for teaching and learning and visualizing, but for larger scale projects, python scripts are more reproducible and easier to run. **Production code** is the code that runs to offer a service to someone or something, and libraries like FASTAI's `nb-dev ` enable you to write whole python libraries with Jupyter notebooks

Usually, in my projects as well, where I do RL + QC , i also use python scripts, where to run the code from the terminal, you have to type something like:


```python
python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS
```

it's much better and modular, and is nicer to use.


Let's begin:

In [11]:
%pip install torch torchvision torchaudio

/home/monitsharma/parallel_adventures/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [12]:



import os
import torch
from torch import nn
import matplotlib.pyplot as plt

# let's see the pytorch version
torch.__version__



'2.7.1+cu126'

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Getting Data

We will load the same data, we had in the earlier notebook:

In [14]:
import os
import requests
import zipfile
from pathlib import Path


# setup path to data folder
data_path = Path("data/")
image_path = data_path/"pizza_steak_sushi"

# if the folder doesn't exist, download and prepare it
if image_path.is_dir():
  print(f"{image_path} directory exusts. ")
else:
  print(f"Did not find {image_path} directory, creating one... ")
  image_path.mkdir(parents=True, exist_ok=True)


# download it
with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading the data")
  f.write(request.content)


# unzip it
with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping the data")
  zip_ref.extractall(image_path)

# remove the zip file
os.remove(data_path/"pizza_steak_sushi.zip")

Did not find data/pizza_steak_sushi directory, creating one... 
Unzipping the data


The folder structure looks like:

```bash
data/
└── pizza_steak_sushi/
    ├── train/
    │   ├── pizza/
    │   │   ├── train_image01.jpeg
    │   │   ├── test_image02.jpeg
    │   │   └── ...
    │   ├── steak/
    │   │   └── ...
    │   └── sushi/
    │       └── ...
    └── test/
        ├── pizza/
        │   ├── test_image01.jpeg
        │   └── test_image02.jpeg
        ├── steak/
        └── sushi/
```

### Create Datasets and DataLoaders (`data_setup.py`)



Once we have the data, we can turn it into a PyTorch `Dataset` and then to `DataLoader`, we can convert the useful `Dataset` and `DataLoader` creating code into a function called `create_dataloaders()`


And we write it to file using the line `%%writefile going_modular/data_setup.py`

In [15]:
%%writefile going_modular/data_setup.py
"""
Contiains the functionallity for creating PyTorch DataLoaders for
image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS):
  """
  Creates training and testing DataLoaders.
  """

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # get the class name
  class_names = train_data.classes

  # turn images into dataloaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True
  )

  return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


Now we can do this:

```python
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=image_path/"train",
    test_dir=image_path/"test",
    transform=data_transform,
    batch_size=32
)

train_dataloader, test_dataloader, class_names
```

### **Making a Model `model_builder.py`**

In [16]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """
  Creates the TinyVGG architecture.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3,stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )


    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
    )


  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Overwriting going_modular/model_builder.py


Now, instead of coding the TinyVGG model from scratch every time, we can just import it using

```python
import torch
from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)
model_1 = model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)
```

### Creating `train_step()` and `test_step()` functions and `train()` to combine them

1. `train_step()` - takes in a model, a `DataLoader`, a loss function and an optimizer and trains the model on the `DataLoader`

2. `test_step()` - takes in a model, a `DataLoader` and a loss function and evaluates the model on the `DataLoader`

3. `train()` - performs 1. and 2. together for a given number of epochs and returns a results dictionary

In [17]:
%%writefile going_modular/engine.py
"""
Contains function for training and testing
"""

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module,
                optimizer: torch.optim.Optimizer,
                device: torch.device) -> Tuple[float, float]:

                model.train()

                train_loss, train_acc = 0, 0

                for batch, (X,y) in enumerate(dataloader):
                  X,y = X.to(device), y.to(device)

                  y_pred = model(X)

                  loss = loss_fn(y_pred, y)
                  train_loss += loss.item()

                  optimizer.zero_grad()

                  loss.backward()

                  optimizer.step()

                  y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                  train_acc += (y_pred_class == y).sum().item()/len(y_pred)

                train_loss = train_loss / len(dataloader)
                train_acc = train_acc / len(dataloader)
                return train_loss, train_acc



def test_step(model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module,
                device: torch.device) -> Tuple[float, float]:

                model.eval()

                test_loss, test_acc = 0, 0

                with torch.inference_mode():
                  for batch, (X,y) in enumerate(dataloader):
                    X,y = X.to(device), y.to(device)

                    test_pred_logits = model(X)

                    loss = loss_fn(test_pred_logits, y)
                    test_loss += loss.item()

                    test_pred_labels = test_pred_logits.argmax(dim=1)
                    test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

                test_loss = test_loss / len(dataloader)
                test_acc = test_acc / len(dataloader)
                return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs:int,
          device: torch.device) -> Dict[str, List]:

          results = {"train_loss": [],
                     "train_acc": [],
                     "test_loss": [],
                     "test_acc": []
                    }


          for epoch in tqdm(range(epochs)):
            train_loss, train_acc = train_step(model=model,
                                                dataloader=train_dataloader,
                                                loss_fn=loss_fn,
                                                optimizer=optimizer,
                                                device=device)

            test_loss, test_acc = test_step(model=model,
                                            dataloader=test_dataloader,
                                            loss_fn=loss_fn,
                                            device=device)


            # print
            print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_acc: {train_acc:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_acc: {test_acc:.4f}"
            )

            # update results dictionary
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            results["test_loss"].append(test_loss)
            results["test_acc"].append(test_acc)

          return results

Overwriting going_modular/engine.py


Now, we have the `engine.py` script, we can make use of

```python
from going_modular import engine
engine.train()
```

### Creating a function to save the model `utils.py`

Often, we want to save our model whilst it's training or after training. it's common practice to store helper functions in a file called `utils.py`

Let's save our model `save_model()` in a file called `utils.py`

In [18]:
%%writefile going_modular/utils.py

import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name:str):

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)


  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name



  # save the model
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

Overwriting going_modular/utils.py


Now, if we wanted to use our `save_model()` function, instead of writing it all over again, we can import it and use it via:

```python
from going_modular import utils
save_model(model=....,
            target_dir=...,
            model_name=...)
```

### Train, evaluate and save the model `train.py`

The file is essentially saying "train the model using whatever data is available"

In our `train.py` file, we'll combine all of the functionality of the other Python scripts we've created and use it totrain a model

In [19]:
from going_modular import data_setup, engine, model_builder, utils

/home/monitsharma/parallel_adventures/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
%%writefile going_modular/train.py

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001



train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

device = "cuda" if torch.cuda.is_available() else "cpu"

# create transform
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])


train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE
)


model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units= HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)


loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

utils.save_model(model=model,
                 target_dir="models",
                 model_name="pytorch_going_modular")

Writing going_modular/train.py


Now, we can just run al this, with `python train.py`